## Data Preprocessing and Data Analysis

In [83]:
import pandas as pd
import numpy as np
import ast

RATING_THRESHOLD = 1000

In [110]:
def preprocess_df(df):
    df = df.copy()
    df.dropna(subset=['author', 'summary', 'genres'], inplace=True)
    df.drop(columns=['about_author', 'community_reviews', 'kindle_price', 'num_reviews', 'id'], inplace=True)
    df['author'] = df['author'].apply(lambda x: ast.literal_eval(x)[0])
    na_authors = ['Anonymous', 'Unknown', 'Various']
    df = df[~df['author'].isin(na_authors)]
    df = df[~df['summary'].str.len() < 75]
    df['genres'] = df['genres'].apply(lambda x: " ".join(ast.literal_eval(x)))
    df['tags'] = df['genres'].str.lower() + ' ' + df['author'].str.lower().replace(' ', '') + ' ' + df['summary'].str.lower()
    df = df[df['num_ratings'] > RATING_THRESHOLD]
    df['w_rating'] = (df['num_ratings'] / (df['num_ratings'] + RATING_THRESHOLD) * df['star_rating'] +
                  RATING_THRESHOLD / (df['num_ratings'] + RATING_THRESHOLD) * df['star_rating'].mean()).round(2)
    return df

In [111]:
new_df = preprocess_df(test_df)